In [14]:
class Variable :
    def __init__(self, data) :
        self.data = data
        
class Function :
    def __call__(self, input) :
        x = input.data
        self.input = x
        y = self.forward(x) # 구체적인 계산은 forward에서 이루어짐
        output = Variable(y)
        return output
    
    def forward(self, x) :
        raise NotImplementedError()
        

class Square(Function) :
    def forward(self, x) :
        return x ** 2

class Exp(Function) :
    def forward(self, x) :
        return np.exp(x)

def numerical_diff(f, x, eps = 1e-4) :
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [17]:
import numpy as np

def f(x) :
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694
